In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores.pgvector import PGVector
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Define the file path for the PDF document
file_path = ''

# Initialize a loader for PDF documents
loader = PyPDFLoader(file_path=file_path)

# Initialize a text splitter for dividing text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Load and split the document
data = loader.load_and_split(text_splitter=text_splitter)

# Define the collection name for storing embeddings
COLLECTION_NAME = "small_chunks"

# Construct the connection string to the PostgreSQL database
CONNECTION_STRING = PGVector.connection_string_from_db_params(                                                  
    driver="psycopg",
    user=os.getenv("PGVECTOR_USER"),
    password=os.getenv("PGVECTOR_PASSWORD"),
    host=os.getenv("PGVECTOR_HOST"),
    port=os.getenv("PGVECTOR_PORT"),
    database=os.getenv("PGVECTOR_DATABASE"),
)

# Initialize the text embedding model
embeddings = BedrockEmbeddings(region_name='us-east-1')

# Create a vector database store instance and populate it with document data and embeddings
db = PGVector.from_documents(
                                documents=data,
                                embedding=embeddings,
                                collection_name=COLLECTION_NAME,
                                connection_string=CONNECTION_STRING
                            )

/Users/sofiia.kalinina/private/it-gkb/.venv/lib/python3.9/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


In [2]:
from langchain_community.chat_models import BedrockChat
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler

# Load environment variables from a .env file
load_dotenv()

prompt = ChatPromptTemplate.from_template("The context, which I am going to provide you, is in German, the question is in English. If you don't know the answer, just say that you don't know, don't try to make up an answer. Answer questions based on the context below: {context} / Question: {question}")


# Initialize the language model
llm = BedrockChat(
    model_id='anthropic.claude-3-sonnet-20240229-v1:0',
    region_name='us-east-1',
    model_kwargs={'max_tokens': 1000}
)

# Set up the retrieval chain with the language model and database retriever
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=db.as_retriever(),
    verbose=True
)


# Initialize the output callback handler
handler = StdOutCallbackHandler()

# Run the retrieval chain with a query
chain.run(
            'What does the document says about resource confilicts on the cluster nodes? how should I avoid them?',
            callbacks=[handler]
        )



/Users/sofiia.kalinina/private/it-gkb/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(
/Users/sofiia.kalinina/private/it-gkb/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: The context, which I am going to provide you, is in German, the question is in English. If you don't know the answer, just say that you don't know, don't try to make up an answer. Answer questions based on the context below: nicht ausreichend getestet ist oder die den Freigabeprozess nicht durchlaufen hat. Auch ist es bei 
Fehlern in den Berechtigungen auf der CI/CD- Umgebung möglich, dass Schadsoftwar e in die Cluster 
eindringen und dort Daten auslesen, löschen oder verändern kann.  
2.5. Unberechtigte Kommunikation  
Alle Pods in einem Cluster sind grundsätzlich in der Lage, miteinander, mit den Nodes im eigenen 
Cluster sowie beliebigen anderen IT -Systemen zu k ommunizieren. Sofern diese Kommunikation nicht 
eingeschränkt ist, kann dies ausgenutzt werden, um z. B. die Control Plane, andere Pods oder die Nodes anzugreifen.  
Auch bes

'Based on the provided context, the document mentions the following about resource conflicts on cluster nodes and how to avoid them:\n\nUnder section 2.3. "Ressourcenkonflikte auf Nodes" (Resource conflicts on nodes), it states:\n\n"Einzelne Pods können den Node oder auch die Orchestrierung überlasten und so die Verfügbarkeit aller anderen Pods auf dem Node oder auch den Betrieb des Nodes selbst gefährden."\n\nTranslation: "Individual pods can overload the node or even the orchestration, thus endangering the availability of all other pods on the node or even the operation of the node itself."\n\nThe document does not explicitly mention how to avoid these resource conflicts, but it provides some recommendations that could help mitigate the risk:\n\n1. Under APP.4.4.A14, it recommends using dedicated nodes for different purposes (bastion, management, storage) to isolate workloads.\n\n2. Under APP.4.4.A15, it recommends using separate Kubernetes clusters or dedicated nodes for application